# This model is not ouputing yet clear images , it is still a working draft

###### lowered thelearning rate for:
###### - generator from 0.0002 to 0.00008
###### - discriminator from 0.002 to  0.001
###### and GAN trainning is considerably faster and debug images considerably better

# Spectrally Normalized Conditional Generative Adversarial Networks (SN-CGAN)


In [ ]:
%pip install ipympl
%pip install torchinfo

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget


import torch
from torch import nn
import torchvision
from torch.utils.data import  DataLoader
import torch.optim as optim

from tqdm.notebook import tqdm_notebook


import time

import numpy as np

from numpy.random import random

In [ ]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)
%cd '/content/drive/My Drive/Colab Notebooks/GANs'

'\nfrom google.colab import drive\n\ndrive.mount("/content/drive", force_remount=True)\n%cd \'/content/drive/My Drive/Colab Notebooks/GANs\'\n'

In [ ]:
%load_ext tensorboard

!nvidia-smi

Fri Nov 25 12:26:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 526.98       Driver Version: 526.98       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   39C    P0    N/A /  N/A |      0MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os

print("Current working directory: {0}".format(os.getcwd()))

Current working directory: c:\Users\androgo\Documents\Python Scripts\myLabProject\GANs_VAE


In [ ]:
from utils.gans.gans_building_blocks import GaussianNoise, PixelwiseNorm, MinibatchStdDev

In [ ]:
image_size=128

The Discriminator and Generator layer for the label conditioning uses an embedding layer to transform the labels into vectors of size embedding_dim ,this expansion of every label (which is anumber) into a vector of size embedding_dim is done to allow the generator to distinguish better the label and to learn with higher probability a different representation for each label.


In [ ]:
class Conditional_GANGenerator(nn.Module):
    """
    noise_dim: the dimension of the noise vector, a scalar
    main : :py:class:`torch.nn.Sequential`
      The sequential container

    """

    def __init__(self, noise_dim=100, condition_label_dim=3):
        """Init function

        Parameters
        ----------
        noise_dim : int
          The dimension of the noise
        conditional_dim : int
          The dimension of the conditioning variable
        channels : int
          The number of channels in the image
        """
        super(Conditional_GANGenerator, self).__init__()
    
        embedding_dim = 100
        
        # output size : torch.Size([32, 3, 25])
        self.label_conditioned_generator = nn.Sequential(
                    nn.Embedding(condition_label_dim, embedding_dim),
                    nn.Linear(embedding_dim, 300),
                    nn.BatchNorm1d(3)
                    )
         
        # layer for the noise used for generation of the image (as is done in the original GAN paper and other GANs)
        #output size :torch.Size([32, 12800])
        out_features=2*image_size*5*5
        self.latent = nn.Sequential(
                    nn.Linear(noise_dim, out_features),
                    #nn.BatchNorm1d(5*5*256),
                    nn.GroupNorm(25*2, out_features),
                    nn.LeakyReLU(0.1, inplace=True)
                    
                    )  
        

        # Build the neural network
        self.gen1 = nn.Sequential(

            # input is Z, going into a convolution
            nn.ConvTranspose2d(
                in_channels=292,
                out_channels=292,
                kernel_size=5, stride=2, padding=1, bias=False
            ),
            nn.BatchNorm2d(292),
            nn.LeakyReLU(0.1,inplace=True),
            PixelwiseNorm()
        )
        
        self.gen2 = nn.Sequential(

            nn.ConvTranspose2d(
                in_channels=292,
                out_channels=256,
                kernel_size=7, stride=1, padding=1, bias=False,
            ),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1,inplace=True),
            nn.PixelShuffle(2),
            PixelwiseNorm()
        )
            
        self.gen3 = nn.Sequential(

            nn.ConvTranspose2d(
                in_channels=64,
                out_channels=64,
                kernel_size=3, stride=2, padding=1, bias=False,
            ),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1,inplace=True),
            PixelwiseNorm()
        )
        
        self.gen4 = nn.Sequential(

            nn.ConvTranspose2d(
                in_channels=64,
                out_channels=32,
                kernel_size=5, stride=1, padding=0, bias=False,
            ),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1,inplace=True),
            PixelwiseNorm()

        )

        self.gen5 = nn.Sequential(

            nn.ConvTranspose2d(
                in_channels=32,
                out_channels=32,
                kernel_size=3, stride=2, padding=0, bias=False,
            ),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1,inplace=True),
            PixelwiseNorm()

        )
         
        self.gen6 = nn.Sequential(

            nn.ConvTranspose2d(
                in_channels=32,
                out_channels=3,
                kernel_size=3, stride=1, padding=0, bias=False
            ),
            nn.Tanh()
        )

    def forward(self, noise, condition_label):
        """Forward function

        Parameters
        ----------
        noise : : noise_tensor :py:class:`torch.Tensor`
         
        condition : : condition_tensor :py:class:`torch.Tensor`
          The conditional one hot encoded vector for the minibatch.

        Returns
        -------
        :py:class:`torch.Tensor`
          the output of the generator (i.e. an image)

        """
        
        noise_vector = noise
        
        label_output = self.label_conditioned_generator(condition_label)
        label_output = label_output.view(label_output.shape[0],-1, 5, 5)  # torch.Size([32, 3, 5, 5])
        
        latent_output = self.latent(noise_vector)
        latent_output = latent_output.view(latent_output.shape[0],-1,5,5) # torch.Size([32, 512, 5, 5])
        
        concat_vector = torch.cat((latent_output, label_output), dim=1)   # torch.Size([32, 515, 5, 5])

        output_gen = self.gen1(concat_vector)
        output_gen = self.gen2(output_gen)
        output_gen = self.gen3(output_gen)
        output_gen = self.gen4(output_gen)
        output_gen     = self.gen5(output_gen)
        output     = self.gen6(output_gen)

        
        return output[:,:,:image_size,:image_size] # output 129 x 129

In [ ]:
class Conditional_GANDiscriminator(nn.Module):
    """Class implementating the conditional GAN discriminator

    Attributes
    ----------
    conditional_dim: int
      The dimension of the conditioning variable.
    channels: int
      The number of channels in the input image (default: 3).
    main : :py:class:`torch.nn.Sequential`
      The sequential container

    """

    def __init__(self, condition_label_dim=3):
        """Init function

        Parameters
        ----------
        conditional_dim: int
          The dimension of the conditioning variable.
        channels: int
          The number of channels in the input image (default: 3).
        """
        super(Conditional_GANDiscriminator, self).__init__()

        embedding_dim = 50

        self.label_condition_disc = nn.Sequential(
                nn.Embedding(condition_label_dim, embedding_dim),
                      nn.Linear(embedding_dim, 6*image_size*image_size),
                      nn.BatchNorm1d(3))
      
      
        self.disc1 = nn.Sequential(

            # Gaussian noise with stdev 0.1 relative to input tensor
            # Salimans et al used it in the output layer of D while OpenAI has it on the input
            GaussianNoise(),

            # input is (nc) x 128 x 128
            nn.utils.spectral_norm(
                nn.Conv2d(
                    in_channels=21,
                    out_channels=128,
                    kernel_size=5, stride=1, padding=0, bias=False
                )
            ),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(0.2)
        )
        
        self.disc2 = nn.Sequential(

            nn.utils.spectral_norm(
                nn.Conv2d(
                    in_channels=128,
                    out_channels=128, #256
                    kernel_size=3,
                    stride=1,
                    padding=0,
                    bias=False,
                )
            ),
            nn.AvgPool2d(2),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(0.2)
        )
        
        self.disc3 = nn.Sequential(

            nn.utils.spectral_norm(
                nn.Conv2d(
                    in_channels=128,   #256
                    out_channels=128,
                    kernel_size=5,
                    stride=1,
                    padding=0,
                    bias=False,
                ) 
            ),
            nn.AvgPool2d(2),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(0.2)
        )
        
        
        self.disc4 = nn.Sequential(

            nn.utils.spectral_norm(
                nn.Conv2d(
                    in_channels=128,
                    out_channels=64,
                    kernel_size=3,
                    stride=1,
                    padding=0,
                    bias=False,
                )
            ),
            nn.AvgPool2d(3),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(0.2)
        )
        
        
        self.disc5 = nn.Sequential(
            nn.utils.spectral_norm(
                nn.Conv2d(
                    in_channels=64,
                    out_channels=32,
                    kernel_size=7, stride=1, padding=1 ,bias=False
                )
            ),
            nn.AvgPool2d(2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(0.2)
        )
        
        self.disc6 = nn.Sequential(
            nn.Conv2d(
                in_channels=32,
                out_channels=1,
                kernel_size=3, stride=1, padding=1, bias=False
            ),
            nn.AvgPool2d(2),
            nn.Dropout(0.2),
            MinibatchStdDev()

            # nn.Sigmoid() # replaced by  nn.BCEWithLogitsLoss()
        )
    

    def forward(self, images, labels_condition):
        """Forward function

        Parameters
        ----------
        images: a batch of images tensors with dimension (im_dim)
       
        condition : : condition_tensor :py:class:`torch.Tensor`
              The corresponding conditional feature maps.
        Returns
        -------
        :py:class:`torch.Tensor`
          the output of the discriminator
        """

        label_output = self.label_condition_disc(labels_condition)
        label_output = label_output.view(label_output.shape[0], -1, image_size, image_size)
        
        input_discriminator = torch.cat((images, label_output), dim=1)


        disc_pred = self.disc1(input_discriminator)
        disc_pred = self.disc2(disc_pred)
        disc_pred = self.disc3(disc_pred)
        disc_pred = self.disc4(disc_pred)
        disc_pred = self.disc5(disc_pred)
        disc_pred = self.disc6(disc_pred)

        return disc_pred

In [ ]:
from utils.utils_cuda import clear_cuda_cache, get_device

from torchinfo import summary

clear_cuda_cache()
DEVICE=get_device()

from utils.utils_cuda import seed_everything

# Hyperparameters
RANDOM_SEED = 42
seed_everything(seed=RANDOM_SEED)



Device: cuda:0
Name of current CUDA device: NVIDIA GeForce MX250


### Test the Generator


In [ ]:
batch_size=4

input_noise = torch.rand(batch_size, 100, dtype=torch.float32, device=DEVICE)
cond_labels = torch.zeros(batch_size, 3, dtype=torch.long, device=DEVICE)


generator = Conditional_GANGenerator(noise_dim=100, condition_label_dim=3).to(DEVICE).eval()

output = generator.forward(input_noise, cond_labels)
print('output generator shape: ',output.shape)

print('GENERATOR ARCHITECTURE: ')
#summary(generator, input_size=((batch_size, 100),(batch_size, 3)), device=DEVICE)
summary(generator, input_data=(input_noise, cond_labels), device=DEVICE)

output generator shape:  torch.Size([4, 3, 128, 128])
GENERATOR ARCHITECTURE: 


Layer (type:depth-idx)                   Output Shape              Param #
Conditional_GANGenerator                 [4, 3, 128, 128]          --
├─Sequential: 1-1                        [4, 3, 300]               --
│    └─Embedding: 2-1                    [4, 3, 100]               300
│    └─Linear: 2-2                       [4, 3, 300]               30,300
│    └─BatchNorm1d: 2-3                  [4, 3, 300]               6
├─Sequential: 1-2                        [4, 6400]                 --
│    └─Linear: 2-4                       [4, 6400]                 646,400
│    └─GroupNorm: 2-5                    [4, 6400]                 12,800
│    └─LeakyReLU: 2-6                    [4, 6400]                 --
├─Sequential: 1-3                        [4, 292, 11, 11]          --
│    └─ConvTranspose2d: 2-7              [4, 292, 11, 11]          2,131,600
│    └─BatchNorm2d: 2-8                  [4, 292, 11, 11]          584
│    └─LeakyReLU: 2-9                    [4, 292, 11, 11]       

### Test the Discriminator


In [ ]:
batch_size=8

# Instantiate Discriminator
discriminator = Conditional_GANDiscriminator(condition_label_dim=5).to(DEVICE).eval()

input_img_discrim = torch.rand(batch_size, 3, 128, 128, dtype=torch.float32,device=DEVICE)
cond_labels = torch.zeros(batch_size, 3,  dtype=torch.long,device=DEVICE)

output = discriminator.forward(input_img_discrim, cond_labels)
print('output discriminator shape: ', output.shape)


print('DISCRIMINATOR ARCHITECTURE: ')
batch_size=32
#summary(discriminator, input_size=((batch_size, 3, 128, 128),(batch_size, 13, 128, 128)), device=DEVICE)
summary(discriminator,input_data=(input_img_discrim, cond_labels), device=DEVICE)

output discriminator shape:  torch.Size([8, 1, 1, 1])
DISCRIMINATOR ARCHITECTURE: 


Layer (type:depth-idx)                   Output Shape              Param #
Conditional_GANDiscriminator             [8, 1, 1, 1]              --
├─Sequential: 1-1                        [8, 3, 98304]             --
│    └─Embedding: 2-1                    [8, 3, 50]                250
│    └─Linear: 2-2                       [8, 3, 98304]             5,013,504
│    └─BatchNorm1d: 2-3                  [8, 3, 98304]             6
├─Sequential: 1-2                        [8, 128, 124, 124]        --
│    └─GaussianNoise: 2-4                [8, 21, 128, 128]         --
│    └─Conv2d: 2-5                       [8, 128, 124, 124]        67,200
│    └─LeakyReLU: 2-6                    [8, 128, 124, 124]        --
│    └─Dropout: 2-7                      [8, 128, 124, 124]        --
├─Sequential: 1-3                        [8, 128, 61, 61]          --
│    └─Conv2d: 2-8                       [8, 128, 122, 122]        147,456
│    └─AvgPool2d: 2-9                    [8, 128, 61, 61]          --

In [ ]:
from utils.gans.gans_statistics_utils import GANStatisticsSaver
from torch.utils.tensorboard import SummaryWriter

from utils.gans.gans_folders import FolderStructure

from pathlib import Path

### Trainning


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import time

import logging

logger = logging.getLogger("CGAN_Trainer")


class ConditionalGAN_Trainer(object):
    """Class to train a Conditional GAN

    Attributes
    ----------
    generator : :py:class:`torch.nn.Module`
      The generator network
    discriminator : :py:class:`torch.nn.Module`
      The discriminator network
    image_size: list of :obj:`int`
      The size of the images in this format: [channels,height, width]
    batch_size: int
      The size of your minibatch
    noise_dim: int
      The dimension of the noise (input to the generator)
    conditional_dim: int
      The dimension of the conditioning variable
    use_gpu: bool
      If you would like to use the gpu
    fixed_noise : :py:class:`torch.Tensor`
      The fixed input noise to the generator.
    fixed_one_hot_label : :py:class:`torch.Tensor`
      The set of fixed one-hot encoded conditioning variable
    criterion : :py:class:`torch.nn.BCELoss`
      The binary cross-entropy loss

    """

    def __init__(
        self,
        net_cond_GANGenerator: Conditional_GANGenerator,
        net_cond_GANDiscriminator: Conditional_GANDiscriminator,
        noise_dim=100,
        conditional_dim=3,
        folder_structure: FolderStructure = FolderStructure(trainning_output = Path('trainings_output') / Path('SNCGAN_training_output'))

    ):
        """Init function

        Parameters
        ----------
        net_cond_GANGenerator : :py:class:`torch.nn.Module`
          The generator network
        net_cond_GANDiscriminator : :py:class:`torch.nn.Module`
          The discriminator network
        image_size: list of :obj:`int`
          The size of the images in this format: [channels,height, width]
        batch_size: int
          The size of your minibatch
        noise_dim: int
          The dimension of the noise (input to the generator)
        conditional_dim: int
          The dimension of the conditioning variable
        use_gpu: bool
          If you would like to use the gpu
        verbosity_level: int
          The level of verbosity output to stdout

        """
        self.net_cond_GANGenerator = net_cond_GANGenerator
        self.net_cond_GANDiscriminator = net_cond_GANDiscriminator
        # binary cross-entropy loss:This loss combines a Sigmoid layer and the BCELoss in one single class.
        self.criterion = nn.BCEWithLogitsLoss()
        
        # move stuff to GPU if needed
        self.net_cond_GANGenerator.to(DEVICE)
        self.net_cond_GANDiscriminator.to(DEVICE)
        self.criterion.to(DEVICE)
        
        self.noise_dim = noise_dim
        self.conditional_dim = conditional_dim

        self.folder_structure = folder_structure
        
        # instantiate tensorboard
        # (tensorboard) writer will output to ./runs/ directory by default
        self.writer = SummaryWriter(log_dir=self.folder_structure.tensorboard_folder)
        #%reload_ext tensorboard
        # %tensorboard --logdir Cond_SNGAN_training_output/Tensorboard_runs
        
        # fixed conditional noise - used to generate samples (one for each value of the conditional variable)
        ''''
        self.fixed_noise = torch.FloatTensor(
            self.conditional_dim, noise_dim, 1, 1
        ).normal_(0, 1)
        self.fixed_one_hot_label = torch.FloatTensor(
            self.conditional_dim, self.conditional_dim, 1, 1
        ).zero_()
        for k in range(self.conditional_dim):
            self.fixed_one_hot_label[k, k] = 1
        '''


    # smoothing labels class=1 to [0.94, 0.98]
    def smooth_positive_labels(self, real_labels):
        real_labels = real_labels.to('cpu')
        labels = real_labels - 0.08 + (random(real_labels.shape) * 0.02)

        return labels.to(DEVICE)
    
    def get_noise(self, batch_size):
        noise = torch.randn(batch_size, self.noise_dim, dtype=torch.float32, device=DEVICE)
        
        #create tensor with random values in range (min, max)
        ''' 
        max = 1
        min = -1
        
        rand_tensor = (max-min)*noise + min
        '''

        return noise
    def get_noise2(self, batch_size):
        noise = torch.tensor((), dtype=torch.float32, device=DEVICE).new_empty((batch_size, self.noise_dim)).uniform_(-1,1)        
        return noise

    def get_conditional_labels(self, batch_size):
          self.fixed_one_hot_label = torch.zeros(batch_size, self.conditional_dim,  dtype=torch.long, device=DEVICE)

          self.fixed_one_hot_label_list = []
          
          for label in range(self.fixed_one_hot_label.shape[1]):
              _tensor=self.fixed_one_hot_label.clone()
              _tensor[:,label] = 1
              self.fixed_one_hot_label_list.append(_tensor)
              
          #labels unset , so the generator will generate simple random images
          self.fixed_one_hot_label_list.append(self.fixed_one_hot_label)
          
          return self.fixed_one_hot_label_list    

    def record_evaluations(self, ganStatisticsSaver, 
                           gener_loss, discr_loss,
                           d_real_loss, d_fake_loss, 
                           train_loader, epoch, batch_idx):
         
        _gener_loss = np.round(gener_loss.item(), 2)
        _discr_loss = np.round(discr_loss.item(), 2)
        # write losses for G and D to tensorboard
        self.writer.add_scalars('Cond_GAN_LOSS', {'GEN': np.round(gener_loss.item(), 2), 
                                                  'DISC': np.round(discr_loss.item(), 2)},
                                len(train_loader) * epoch + batch_idx + 1)

        ganStatisticsSaver.save_train_generator_loss_per_batch(_gener_loss)
        ganStatisticsSaver.save_train_discriminator_loss_per_batch(_discr_loss)

        # write losses for G and D to tensorboard
        self.writer.add_scalars('DISC_LOSS', {'DISC_REAL': np.round(d_real_loss.item(), 2),
                                                'DISC_FAKE': np.round(d_fake_loss.item(), 2)},
                                len(train_loader) * epoch + batch_idx + 1)

    def save_images(self,model:nn.Module, fixed_one_hot_label_list,labels, ganStatisticsSaver,train_loader,epoch,batch_idx,is_debug=False):
        model.eval()
        log_list_generated_images = []
        # generate the debug imagery on current generator model for every 
        with torch.no_grad():
          for cond_label in fixed_one_hot_label_list:
              log_generated_images = model(self.get_noise(self.batch_size),
                                                  cond_label)            
              log_list_generated_images.append(log_generated_images.detach().cpu())
          
          log_generated_images = model(self.get_noise(self.batch_size),
                                                  labels)     
        
        log_list_generated_images.append(log_generated_images.detach().cpu())
        
        log_list_generated_images = torch.cat(log_list_generated_images, dim=0) 
        # make grid of 5x5 generated images to get an idea of G's performance
        list_intermediate_imagery_grid = torchvision.utils.make_grid(
            log_list_generated_images, nrow=self.batch_size, padding=2, normalize = True)
        
        model.train()

        
        #add epoch images
        if not is_debug:
          id = str(epoch)
          ganStatisticsSaver.save_images_normalized_to_folder(
                      log_list_generated_images, 
                      self.folder_structure.trainning_images_generated,
                      id)
          
          self.writer.add_image('intermediate generated imagery',
                                list_intermediate_imagery_grid, len(train_loader) * epoch  + 1)
        #add debug images
        else:
          id = str(epoch) + '_' + str(batch_idx)
          ganStatisticsSaver.save_images_normalized_to_folder(
                      log_list_generated_images, 
                      self.folder_structure.debug_images_generated,
                      id)
      
    def train(
        self, 
        train_loader:DataLoader, 
        ganStatisticsSaver: GANStatisticsSaver,
        optimizer_discriminator:optim.Optimizer,
        optimizer_generator:optim.Optimizer,
        epochs_range=(0, 15)
    ):
        """trains the Conditional GAN.

        Parameters
        ----------
        train_loader: :py:class:`torch.utils.data.train_loader`
          The train_loader for your data
        epochs_range: tuple of `int`
          The first element is the starting epoch and the second element is the ending epoch (exclusive) for training the model (e.g. (0, 10) will train the model for 10 epochs,(10, 15) will train the model for 5 epochs more than the previous training)
     
        """
        
        image, _ = next(iter(train_loader))
        batch_size = image.shape[0]
        self.batch_size=batch_size
        fixed_one_hot_label_list = self.get_conditional_labels(batch_size)


        with torch.no_grad():
          self.net_cond_GANGenerator.eval()
          self.net_cond_GANDiscriminator.eval()
          
          self.writer.add_graph(self.net_cond_GANGenerator, 
                                (self.get_noise(batch_size),
                                fixed_one_hot_label_list[2]))
          
          image = image.to(DEVICE)
          self.writer.add_graph(self.net_cond_GANDiscriminator, 
                                (image, fixed_one_hot_label_list[2]))

        self.net_cond_GANGenerator.train()
        self.net_cond_GANDiscriminator.train()


        #the dataloader is modified to have batches of fixed size , so is safe to use these outside the loop
        # smooth the real label = 1
        real_labels = self.smooth_positive_labels(torch.ones(batch_size, device=DEVICE))
        fake_labels = torch.zeros(batch_size,1,1,1, device=DEVICE)  


        start_training_time = time.time()

        for epoch in range(epochs_range[0], epochs_range[1]):
            clear_cuda_cache()

            ganStatisticsSaver.discriminator_loss_real[epoch] = []
            ganStatisticsSaver.discriminator_loss_fake[epoch] = []
            ganStatisticsSaver.generator_loss_fake[epoch] = []
                      
            #for batch_idx, (features, labels) in enumerate(tqdm.tqdm(train_loader), 0):
            for batch_idx, (features, labels) in enumerate(tqdm_notebook(iterable=train_loader, desc='epoch '+str(epoch), total=len(train_loader)), 0):

                # real images
                real_images = features.to(DEVICE, non_blocking=True)
                
                labels = labels.pin_memory() 
                labels = labels.to(DEVICE, non_blocking=True).long()
                labels = labels.squeeze()
                     
                self.net_cond_GANDiscriminator.zero_grad()
    
                discr_pred_real = self.net_cond_GANDiscriminator(real_images, labels)
                
                real_labels=real_labels.view_as(discr_pred_real)
                d_real_loss = self.criterion(discr_pred_real, real_labels)


                # =========
                # GENERATOR
                # =========
                # generated (fake) images
                noise = self.get_noise(batch_size)
                
                gen_images = self.net_cond_GANGenerator(noise, labels)
                discr_pred_fake = self.net_cond_GANDiscriminator(gen_images.detach(), labels)
                d_fake_loss = self.criterion(discr_pred_fake,  fake_labels)
     
                # train with fake
                #D_fake_loss.backward()
              
                discr_loss = d_real_loss + d_fake_loss
                # track discrimantor failure : if the discriminator loss is NAN or zero then the discriminator is not learning 
                assert not torch.isnan(discr_loss).any()
                assert  discr_loss.all()
                
                discr_loss.backward()
                optimizer_discriminator.step()
        

                # =============
                # DISCRIMINATOR
                # =============
        
                # Train generator with real labels
                self.net_cond_GANGenerator.zero_grad()
                
                noise = self.get_noise(batch_size)
                #try disable
                gen_images = self.net_cond_GANGenerator(noise, labels)
                
                d_out_gen = self.net_cond_GANDiscriminator(gen_images, labels) 
                gener_loss = self.criterion(d_out_gen, real_labels)
                
                gener_loss.backward()
                optimizer_generator.step()
                           
                self.record_evaluations(ganStatisticsSaver, 
                                        gener_loss, discr_loss,
                                        d_real_loss, d_fake_loss, 
                                        train_loader, epoch, batch_idx)
              
                #debug images
                if (batch_idx % int(len(train_loader)/4) == 0):
                  with torch.no_grad():
                    self.save_images(self.net_cond_GANGenerator,fixed_one_hot_label_list,labels,
                                     ganStatisticsSaver,train_loader,epoch,batch_idx,
                                     is_debug=True)
              
              
            with torch.no_grad():
              self.save_images(self.net_cond_GANGenerator,fixed_one_hot_label_list,labels,
                                     ganStatisticsSaver,train_loader,epoch,batch_idx,
                                     is_debug=False)
              
              
            # save a checkpoint of the DCGAN every epoch
            # performance often degrades past a certain epoch and may not recover
            # save generator
            torch.save({
                'model_state_dict': self.net_cond_GANGenerator.state_dict(),
                'optimizer_state_dict': optimizer_generator.state_dict(),
                'epoch': epoch,
                'loss': gener_loss.item(),
            }, self.folder_structure.get_path_checkpoints_for_file( 'generator_checkpoint_epoch_' 
                                                      + str(epoch) + '.pt')) 
            
            # save discriminator
            torch.save({
                'model_state_dict': self.net_cond_GANDiscriminator.state_dict(),
                'optimizer_state_dict': optimizer_discriminator.state_dict(),
                'epoch': epoch,
                'loss': discr_loss.item(),
            }, self.folder_structure.get_path_checkpoints_for_file('discriminator_checkpoint_epoch_'
        
                                                      + str(epoch) + '.pt'))
        end_training_time = time.time()

        total_training_time = end_training_time - start_training_time

        ganStatisticsSaver.save_total_training_time(total_training_time)

        ganStatisticsSaver.set_trainned_model(self.net_cond_GANGenerator, self.net_cond_GANDiscriminator)
        ganStatisticsSaver.set_trainned_optimizer(optimizer_generator, optimizer_discriminator)

        ganStatisticsSaver.save_model_params(
            self.folder_structure.trainning_final_model_folder, epoch
        )
      
    
        self.writer.flush()
        self.writer.close()

### Load the dataset

In [ ]:
from utils.gans.utils_gan import weights_init
from utils_data.utils_data_load import get_celeba_datasets_with_labels
from utils.utils_cuda import clear_cuda_cache, get_device
DEVICE=get_device()


# during training the cuda device is working almost 100 %
BATCH_SIZE = 32
train_loader = get_celeba_datasets_with_labels(image_size=128,batch_size=BATCH_SIZE, desired_attr = ['Male', 'Brown_Hair','Eyeglasses'])

print('train loader lenghth', len(train_loader))
images, labels = next(iter(train_loader))
print('images shape: ', images.shape)
print('labels shape: ', labels.shape)
print('labels: ' )

torch.cuda.init()
clear_cuda_cache()


Device: cuda:0
Name of current CUDA device: NVIDIA GeForce MX250
train loader lenghth 545
images shape:  torch.Size([8, 3, 128, 128])
labels shape:  torch.Size([8, 3, 1, 1])
labels: 


### GANs init utils

In [ ]:
def get_GANs_component_models():
    generator = Conditional_GANGenerator()
    generator.apply(weights_init)
    generator.to(DEVICE)

    discriminator = Conditional_GANDiscriminator()
    discriminator.apply(weights_init)
    discriminator.to(DEVICE)
    
    return generator, discriminator

#learning_rate_generator = 0.000001
#learning_rate_discriminator = 0.0009
def get_GANs_optimizers(generator:nn.Module, discriminator:nn.Module,
                    learning_rate_generator = 0.001, learning_rate_discriminator = 0.01):

    
    optimizer_discriminator = optim.AdamW(
        discriminator.parameters(),
        lr=learning_rate_discriminator,
        betas=(0.5, 0.999)
    )

    optimizer_generator = optim.AdamW(
        generator.parameters(),
        lr=learning_rate_generator,
        betas=(0.5, 0.999),
    )
    
    return optimizer_discriminator, optimizer_generator

### Start trainning the GANs models


In [ ]:

generator, discriminator = get_GANs_component_models()
optimizer_discriminator, optimizer_generator = get_GANs_optimizers(generator, discriminator)


folder_structure = FolderStructure(trainning_output = Path('trainings_output') / Path('SNCGAN_training_output'))

# remove prevoius training output if training is not restarted from a checkpoint 
REMOVE_PREVIOUSE_TRAINNING_OUTPUT = True

if REMOVE_PREVIOUSE_TRAINNING_OUTPUT:
    folder_structure.remove_trainning_folder_output()
    
    #create or clear folders if training is restarted or training models are not loaded
    folder_structure.create_folders()
    folder_structure.clean_folders()



trainer = ConditionalGAN_Trainer(
    generator, discriminator, folder_structure = folder_structure
)

ganStatisticsSaver = GANStatisticsSaver()

NUM_EPOCHS_END = 30
NUM_EPOCHS_START = 0

# use main method for lunch if dataloader has the number of jobs set
if __name__ == '__main__':
    trainer.train(train_loader, ganStatisticsSaver,
                  optimizer_discriminator, optimizer_generator, 
                  epochs_range=(NUM_EPOCHS_START, NUM_EPOCHS_END),
                  )
    trainer = None


# use next  for lunch if dataloader does not have the number of jobs set
''' 
NUM_EPOCHS = 2
trainer.train(train_loader, ganStatisticsSaver, n_epochs=NUM_EPOCHS)
'''

epoch 0:   0%|          | 0/545 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Start trainning the GANs from a loaded model

In [ ]:
def train_loaded_model(train_loader, NUMBER_EPOCHS_TO_TRAIN):
    
    generator, discriminator = get_GANs_component_models()
    optimizer_discriminator, optimizer_generator = get_GANs_optimizers(generator,discriminator)

    folder_structure = FolderStructure(trainning_output =Path('trainings_output') / Path('SNCGAN_training_output'))

    ganStatisticsSaver = GANStatisticsSaver()
    last_epoch = ganStatisticsSaver.load_trainned_models(generator, discriminator, 
                                            optimizer_generator, 
                                            optimizer_discriminator,
                                            folder_structure.trainning_final_model_folder)       
    
    trainer = ConditionalGAN_Trainer(
        generator, discriminator, folder_structure = folder_structure
    )
    
    
    NUM_EPOCHS_START = last_epoch + 1
    NUM_EPOCHS_END = NUM_EPOCHS_START + NUMBER_EPOCHS_TO_TRAIN

    trainer.train(train_loader, ganStatisticsSaver, 
                optimizer_discriminator, optimizer_generator, 
                  epochs_range=(NUM_EPOCHS_START, NUM_EPOCHS_END))
    
    
    return generator, folder_structure, ganStatisticsSaver, NUM_EPOCHS_END
    
#se to True if you want to train from a checkpoint 
#START_TRAINNING_FROM_CHECKPOINT = False
START_TRAINNING_FROM_CHECKPOINT = True


if START_TRAINNING_FROM_CHECKPOINT:
    NUMBER_EPOCHS_TO_TRAIN=4
    generator, folder_structure, ganStatisticsSaver,NUM_EPOCHS_END = train_loaded_model(train_loader, NUMBER_EPOCHS_TO_TRAIN)
    

: 

### Use Tensorboard to vizualiza all training combines, below is displayed only for current trainning

In [ ]:
NUM_EPOCHS = NUM_EPOCHS_END + 1

from utils.utils_plotting import (
    plot_multiple_trainning_evals,
    plot_trainning_eval

)

plot_multiple_trainning_evals(
    evals_list=(
        ganStatisticsSaver.train_discriminator_loss_per_batch,
        ganStatisticsSaver.train_generator_loss_per_batch,
    ),
    num_epochs=NUM_EPOCHS,
    custom_labels_list=(" -- Discriminator", " -- Generator"),
)

plot_trainning_eval(
    ganStatisticsSaver.train_discriminator_loss_per_batch,
    num_epochs=NUM_EPOCHS,
    type_plot="Loss",
    custom_label=" Discriminator",
)

plot_trainning_eval(
    ganStatisticsSaver.train_generator_loss_per_batch,
    num_epochs=NUM_EPOCHS,
    type_plot="Loss",
    custom_label=" Generator",
)

: 

In [ ]:
from utils_data.utils_folders import write_to_disk_images_stacked

resulting_images = write_to_disk_images_stacked(folder_source_imgs=folder_structure.trainning_images_generated,
                             folder_output_batches_stacked=folder_structure.stacks_images_generated,
                             folder_output_all_stacked=folder_structure.one_stack_images_generated,
                             num_epochs=NUM_EPOCHS)

: 

In [ ]:
from utils.utils_images import display_images

# plot images saved in folder in a stack of images 
%matplotlib inline

display_images(resulting_images)

: 

### Interpolate images

In [ ]:
#DEVICE='cpu'
DEVICE

: 

In [ ]:
from utils.gans.interpolate_images import interpolate_images

generator.eval() # set generator to evaluation mode 
interpolate_images(folder_root = folder_structure.trainning_model_interpolated_images,
                      DEVICE=DEVICE, generator=generator)

: 